In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std

from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

from scipy.stats import mode

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#npz files can only be loaded using /gdrive path
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [ ]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 7.0 MB/s 


In [ ]:
#import pyeeg
from tensorflow import keras 

In [ ]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [ ]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings 
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100
from keras.utils import np_utils

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way.csv',header=None)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way_sub_482_trial_3.csv',header=None) #the transfer set + subject specific train data , inc semi sup tri-trained labels obtained the keras way (train_on_batch)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119169,-0.002359,0.139156,-0.008791,0.067970,0.106136,0.143015,0.113426,0.005789,0.059822,0.031948,...,-0.274666,-0.113855,-0.155022,0.121145,-0.032377,0.225350,-0.048672,-0.414036,-0.345852,0
119170,-0.136259,-0.109391,-0.123891,-0.130714,-0.159288,-0.149053,-0.161847,-0.102141,-0.091905,-0.026228,...,-0.175068,-0.268039,-0.137538,-0.193406,-0.041581,-0.178906,-0.052243,-0.233495,-0.170376,0
119171,0.016081,0.049101,-0.076117,-0.019083,-0.055534,-0.042669,-0.136583,-0.041811,-0.082550,-0.010078,...,-0.121573,-0.110424,-0.085123,-0.070114,-0.073544,-0.042240,-0.027231,0.014366,-0.018225,0
119172,-0.009220,-0.006647,-0.006218,-0.010506,0.011364,-0.017368,-0.025515,-0.032377,-0.011793,0.010935,...,-0.045242,-0.006218,0.029804,0.075260,0.103991,0.104420,0.073544,0.057678,0.033663,2


In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 14.7 MB/s 


In [ ]:
x_dt=transfer_set_with_tri_trained_sub_labeled_dt.drop([3000],axis=1)
y_dt=transfer_set_with_tri_trained_sub_labeled_dt[3000]

In [ ]:
y_dt_=np_utils.to_categorical(y_dt)

In [ ]:
y_dt_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
x_dt.shape

(119174, 3000)

In [ ]:
x_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119169,-0.002359,0.139156,-0.008791,0.067970,0.106136,0.143015,0.113426,0.005789,0.059822,0.031948,...,-0.052532,-0.274666,-0.113855,-0.155022,0.121145,-0.032377,0.225350,-0.048672,-0.414036,-0.345852
119170,-0.136259,-0.109391,-0.123891,-0.130714,-0.159288,-0.149053,-0.161847,-0.102141,-0.091905,-0.026228,...,-0.323481,-0.175068,-0.268039,-0.137538,-0.193406,-0.041581,-0.178906,-0.052243,-0.233495,-0.170376
119171,0.016081,0.049101,-0.076117,-0.019083,-0.055534,-0.042669,-0.136583,-0.041811,-0.082550,-0.010078,...,-0.097559,-0.121573,-0.110424,-0.085123,-0.070114,-0.073544,-0.042240,-0.027231,0.014366,-0.018225
119172,-0.009220,-0.006647,-0.006218,-0.010506,0.011364,-0.017368,-0.025515,-0.032377,-0.011793,0.010935,...,-0.072258,-0.045242,-0.006218,0.029804,0.075260,0.103991,0.104420,0.073544,0.057678,0.033663


In [ ]:
x_dt_rshp=np.array(x_dt).reshape(119174,3000,1)

In [ ]:
x_dt_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[ 0.01608114],
        [ 0.0491011 ],
        [-0.07611742],
        ...,
        [-0.02723074],
        [ 0.01436582],
        [-0.0182253 ]],

       [[-0.00921986],
        [-0.00664687],
        [-0.00621804],
        ...,
        [ 0.07354444],
        [ 0.05767771],
        [ 0.0336632 ]],

       [[ 0.00788977],
        [-0.07996394],
        [-0.12133194],
        ...,
        [ 0.13668502],
        [ 0.14862629],
        [ 0.10427297]]])

In [ ]:
import numpy as np
np.random.seed(999)

import tensorflow
tensorflow.random.set_seed(999)

import random
#random.seed(999)

In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-j2i63ve5
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-j2i63ve5
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=c06cf06ff0440a90798b6a3af3fca809f3bae6ce4ae017026221bd0e52d360ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-9b82zlp1/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100

In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)
    
    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)
    
    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

In [ ]:
import keras_tuner as kt
optimizer=Adam(lr=1e-4)

In [ ]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
      model=model_b(verbose=VBS)
      optimizer = Adam(lr=1e-4)
      model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
      optimizer = Adam(lr=1e-4)
      return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            x_dt_rshp, y_dt_,
            epochs=hp.Choice("epochs", [40, 50, 60, 70])
        )

In [ ]:
tuner = kt.RandomSearch(
    MyHyperModel(),
    objective="loss",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    #directory="my_dir",
    #project_name="tune_hypermodel",
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
tuner.search(x_dt_rshp, y_dt_)

Trial 3 Complete [00h 50m 40s]
loss: 0.07310095801949501

Best loss So Far: 0.06396068632602692
Total elapsed time: 02h 29m 36s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
epochs: 60
Score: 0.06396068632602692
Trial summary
Hyperparameters:
epochs: 50
Score: 0.07310095801949501
Trial summary
Hyperparameters:
epochs: 40
Score: 0.0862153060734272


Trial 1:

tuner.search() O/P:

Trial 3 Complete [01h 13m 06s]

loss: 0.06341969966888428

Best loss So Far: 0.06341969966888428

Total elapsed time: 03h 02m 59s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary()

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7fdd185aa050>

Trial summary

Hyperparameters:

epochs: 60

Score: 0.06341969966888428

Trial summary

Hyperparameters:

epochs: 50

Score: 0.07388429343700409

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08759897947311401

Trial 2:

tuner.search() O/P:

Trial 3 Complete [01h 02m 14s]

loss: 0.07305266708135605

Best loss So Far: 0.06365969590842724

Total elapsed time: 03h 05m 40s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary():

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7f2632cba5d0>

Trial summary

Hyperparameters:

epochs: 60

Score: 0.06365969590842724

Trial summary

Hyperparameters:

epochs: 50

Score: 0.07305266708135605

Trial summary

Hyperparameters:

epochs: 40

Score: 0.0875231996178627

Trial 3:

tuner.search() O/P:

Trial 3 Complete [00h 50m 40s]

loss: 0.07310095801949501

Best loss So Far: 0.06396068632602692

Total elapsed time: 02h 29m 36s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary() O/P:

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7f865b012690>

Trial summary

Hyperparameters:

epochs: 60

Score: 0.06396068632602692

Trial summary

Hyperparameters:

epochs: 50

Score: 0.07310095801949501

Trial summary

Hyperparameters:

epochs: 40

Score: 0.0862153060734272


Training the teacher model for 40 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
teacher_40_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_40_epochs.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_40_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_teacher_model_40_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_40_epochs_dir = os.path.dirname(checkpoint_teacher_model_40_epochs)

In [ ]:
cp_callback_teacher_model_40_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_40_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=40)

In [ ]:
teacher_40_epochs.fit(x_dt_rshp, y_dt_, epochs=40, callbacks=[cp_callback_teacher_model_40_epochs])

Epoch 1/40
3725/3725 [==============================] - 45s 8ms/step - loss: 0.6869 - accuracy: 0.7230
Epoch 2/40
3725/3725 [==============================] - 29s 8ms/step - loss: 0.4055 - accuracy: 0.8402
Epoch 3/40
3725/3725 [==============================] - 29s 8ms/step - loss: 0.3322 - accuracy: 0.8681
Epoch 4/40
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2861 - accuracy: 0.8862
Epoch 5/40
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2563 - accuracy: 0.8990
Epoch 6/40
3725/3725 [==============================] - 30s 8ms/step - loss: 0.2307 - accuracy: 0.9085
Epoch 7/40
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2164 - accuracy: 0.9156
Epoch 8/40
3725/3725 [==============================] - 29s 8ms/step - loss: 0.1994 - accuracy: 0.9211
Epoch 9/40
3725/3725 [==============================] - 30s 8ms/step - loss: 0.1896 - accuracy: 0.9253
Epoch 10/40
3725/3725 [==============================] - 29s 8ms/step - l

In [ ]:
latest_teacher_40_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_40_epochs_dir)

In [ ]:
print(latest_teacher_40_epochs)
print(checkpoint_teacher_model_40_epochs)

/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_wts_trial_3.ckpt
/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_wts_trial_3.ckpt


In [ ]:
sub_test_data=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [ ]:
sub_test_data.shape

(614, 3001)

In [ ]:
x_test_dt=sub_test_data.drop([3000],axis=1)
y_test_dt=sub_test_data[3000]

In [ ]:
y_test_dt_=np_utils.to_categorical(y_test_dt)

In [ ]:
y_test_dt_

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
x_test_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.115570,-0.115570,-0.110424,-0.144301,-0.218060,-0.275952,-0.265661,-0.247221,-0.216345,-0.156309,...,0.083836,0.090269,0.088982,0.078262,0.066254,0.066683,0.071829,0.099274,0.128006,0.128006
1,-0.001501,0.005360,-0.018654,-0.033234,-0.028088,-0.041811,-0.048243,-0.054676,-0.057678,-0.021227,...,-0.026802,-0.002359,0.026373,0.025087,0.042669,0.040953,0.019512,0.001501,-0.019941,-0.025515
2,-0.217289,-0.202789,-0.189994,-0.138391,-0.105552,-0.050111,0.061199,0.103420,0.173788,0.111097,...,0.054375,0.042861,-0.011728,0.003625,0.010022,0.015140,-0.043714,-0.021110,-0.042008,-0.064184
3,-0.003199,-0.100008,-0.153744,-0.026228,-0.022816,-0.045420,-0.077832,0.006184,-0.007463,-0.088493,...,0.019831,-0.040302,-0.027508,-0.043287,0.052243,-0.037743,-0.082096,-0.086361,-0.076979,-0.159715
4,-0.015140,0.006610,0.049684,0.073993,0.068449,0.046272,0.063331,0.069302,-0.001919,-0.024096,...,0.131141,0.113229,0.070155,0.035184,0.021110,0.023669,-0.008316,-0.006610,-0.000213,-0.024949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.085082,0.089773,0.111097,0.126023,0.126023,0.118347,0.124744,0.136685,0.173788,0.195538,...,-0.049258,-0.036037,-0.003625,0.048831,0.037317,0.034331,0.002346,0.039449,0.043287,0.039875
610,0.003199,0.078258,0.191274,0.130288,0.138817,0.198524,0.181465,0.239465,0.233068,0.259936,...,-0.048405,0.042434,-0.030066,-0.125597,-0.037743,0.168244,-0.060773,-0.009596,-0.050537,-0.116214
611,0.112376,0.032625,0.057361,0.103420,0.048831,-0.019405,0.046699,0.028361,0.078685,0.061626,...,0.022816,0.018125,0.028361,0.088920,0.008743,-0.042008,0.085935,-0.185303,-0.135406,-0.201936
612,0.018225,0.008362,-0.019083,-0.067112,-0.049530,0.017368,0.037523,0.047386,0.026373,-0.008791,...,0.018654,-0.003216,0.004074,0.006218,-0.018225,-0.025944,-0.013508,0.011793,-0.036665,-0.057249


In [ ]:
x_test_dt.shape

(614, 3000)

In [ ]:
x_test_dt_rshp=np.array(x_test_dt).reshape(614,3000,1)

In [ ]:
x_test_dt_rshp

array([[[-1.15569822e-01],
        [-1.15569822e-01],
        [-1.10423861e-01],
        ...,
        [ 9.92742659e-02],
        [ 1.28005908e-01],
        [ 1.28005908e-01]],

       [[-1.50090688e-03],
        [ 5.36038176e-03],
        [-1.86541271e-02],
        ...,
        [ 1.50090688e-03],
        [-1.99406186e-02],
        [-2.55154162e-02]],

       [[-2.17288668e-01],
        [-2.02788530e-01],
        [-1.89994296e-01],
        ...,
        [-2.11104773e-02],
        [-4.20077189e-02],
        [-6.41843840e-02]],

       ...,

       [[ 1.12375984e-01],
        [ 3.26252836e-02],
        [ 5.73607955e-02],
        ...,
        [-1.85303083e-01],
        [-1.35405597e-01],
        [-2.01935587e-01]],

       [[ 1.82252966e-02],
        [ 8.36219523e-03],
        [-1.90829576e-02],
        ...,
        [ 1.17928392e-02],
        [-3.66650091e-02],
        [-5.72488751e-02]],

       [[-1.38604151e-02],
        [-2.23899008e-02],
        [-6.33314366e-02],
        ...,
        

In [ ]:
accuracy_teacher_model_40_epochs = teacher_40_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 1s 4ms/step - loss: 0.8506 - accuracy: 0.8648


Training the teacher model for 50 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_50_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs_keras_way/teacher_50_epochs_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_50_epochs_dir = os.path.dirname(checkpoint_teacher_model_50_epochs)

In [ ]:
cp_callback_teacher_model_50_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_50_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=50)

In [ ]:
teacher_50_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_50_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_50_epochs.fit(x_dt_rshp, y_dt_, epochs=50, callbacks=[cp_callback_teacher_model_50_epochs])

Epoch 1/50
3725/3725 [==============================] - 45s 8ms/step - loss: 0.6855 - accuracy: 0.7243
Epoch 2/50
3725/3725 [==============================] - 29s 8ms/step - loss: 0.4078 - accuracy: 0.8393
Epoch 3/50
3725/3725 [==============================] - 29s 8ms/step - loss: 0.3371 - accuracy: 0.8656
Epoch 4/50
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2911 - accuracy: 0.8845
Epoch 5/50
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2593 - accuracy: 0.8978
Epoch 6/50
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2345 - accuracy: 0.9074
Epoch 7/50
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2176 - accuracy: 0.9144
Epoch 8/50
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2018 - accuracy: 0.9193
Epoch 9/50
3725/3725 [==============================] - 29s 8ms/step - loss: 0.1917 - accuracy: 0.9239
Epoch 10/50
3725/3725 [==============================] - 29s 8ms/step - l

In [ ]:
latest_teacher_50_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_50_epochs_dir)

In [ ]:
accuracy_teacher_model_50_epochs = teacher_50_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 1s 5ms/step - loss: 0.9905 - accuracy: 0.8567


Training the teacher model for 60 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_60_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_60_epochs_dir = os.path.dirname(checkpoint_teacher_model_60_epochs)

In [ ]:
cp_callback_teacher_model_60_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_60_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=60)

In [ ]:
teacher_60_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_60_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_60_epochs.fit(x_dt_rshp, y_dt_, epochs=60, callbacks=[cp_callback_teacher_model_60_epochs])

Epoch 1/60
3725/3725 [==============================] - 44s 8ms/step - loss: 0.6854 - accuracy: 0.7236
Epoch 2/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.4052 - accuracy: 0.8414
Epoch 3/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.3323 - accuracy: 0.8677
Epoch 4/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2867 - accuracy: 0.8861
Epoch 5/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2569 - accuracy: 0.8994
Epoch 6/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2331 - accuracy: 0.9075
Epoch 7/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2174 - accuracy: 0.9136
Epoch 8/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2016 - accuracy: 0.9201
Epoch 9/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.1909 - accuracy: 0.9244
Epoch 10/60
3725/3725 [==============================] - 29s 8ms/step - l

In [ ]:
latest_teacher_60_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_60_epochs_dir)

In [ ]:
accuracy_teacher_model_60_epochs = teacher_60_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 1s 5ms/step - loss: 0.9506 - accuracy: 0.8502


KD using 60 epochs teacher, 70 epochs student, alpha: 0.9, temp: 5-->Trial 1

KD using 60 epochs teacher, 70 epochs student , alpha: 0.9, temp: 5 ---->trial 2

KD using  epochs teacher,  epochs student , alpha: 0.9, temp: 5 ---->trial 3

In [ ]:
teacher_model=model_b(verbose=VBS) #40 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 12s 8ms/step - loss: 0.8506 - accuracy: 0.8648


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.94202899, 0.13157895, 0.95      , 0.83606557, 0.79746835])

In [ ]:
print(classification_report(y_test_dt, preds_teacher_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.955882  0.942029  0.948905       276
           1   0.333333  0.131579  0.188679        38
           2   0.783505  0.950000  0.858757       160
           3   0.944444  0.836066  0.886957        61
           4   0.797468  0.797468  0.797468        79

    accuracy                       0.864821       614
   macro avg   0.762927  0.731428  0.736153       614
weighted avg   0.850916  0.864821  0.852725       614



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170   

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=50)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/50
3725/3725 [==============================] - 34s 8ms/step - categorical_accuracy: 0.7046 - student_loss: 0.7192 - distillation_loss: 0.0015
Epoch 2/50
3725/3725 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7710 - student_loss: 0.5504 - distillation_loss: 0.0012
Epoch 3/50
3725/3725 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7876 - student_loss: 0.5117 - distillation_loss: 0.0011
Epoch 4/50
3725/3725 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7970 - student_loss: 0.4936 - distillation_loss: 0.0010
Epoch 5/50
3725/3725 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8014 - student_loss: 0.4840 - distillation_loss: 0.0010
Epoch 6/50
3725/3725 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8060 - student_loss: 0.4706 - distillation_loss: 9.8461e-04
Epoch 7/50
3725/3725 [==============================] - 29s 8ms/step - categorical_accurac

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 0s 3ms/step - categorical_accuracy: 0.8355 - student_loss: 0.8070


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
print(classification_report(y_test_dt, preds_student_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.946237  0.956522  0.951351       276
           1   0.250000  0.052632  0.086957        38
           2   0.719212  0.912500  0.804408       160
           3   0.811594  0.918033  0.861538        61
           4   0.818182  0.569620  0.671642        79

    accuracy                       0.835505       614
   macro avg   0.709045  0.681861  0.675179       614
weighted avg   0.814135  0.835505  0.814652       614



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.95652174, 0.05263158, 0.9125    , 0.91803279, 0.56962025])

In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
#50 epochs teacher

In [ ]:
teacher_model=model_b(verbose=VBS) #50 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs_keras_way/teacher_50_epochs_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 12s 8ms/step - loss: 0.9905 - accuracy: 0.8567


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.9384058 , 0.15789474, 0.975     , 0.68852459, 0.79746835])

In [ ]:
print(classification_report(y_test_dt, preds_teacher_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.966418  0.938406  0.952206       276
           1   0.461538  0.157895  0.235294        38
           2   0.718894  0.975000  0.827586       160
           3   0.976744  0.688525  0.807692        61
           4   0.863014  0.797468  0.828947        79

    accuracy                       0.856678       614
   macro avg   0.797322  0.711459  0.730345       614
weighted avg   0.858391  0.856678  0.845146       614



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 2995, 4)           28        
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 1497, 4)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_8 (Conv1D)           (None, 2995, 4)           28        
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 1497, 4)          0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=60)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/60
3725/3725 [==============================] - 34s 8ms/step - categorical_accuracy: 0.7096 - student_loss: 0.7175 - distillation_loss: 0.0015
Epoch 2/60
3725/3725 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7743 - student_loss: 0.5450 - distillation_loss: 0.0012
Epoch 3/60
3725/3725 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7878 - student_loss: 0.5111 - distillation_loss: 0.0011
Epoch 4/60
3725/3725 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7983 - student_loss: 0.4880 - distillation_loss: 0.0011
Epoch 5/60
3725/3725 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8092 - student_loss: 0.4675 - distillation_loss: 0.0010
Epoch 6/60
3725/3725 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8183 - student_loss: 0.4469 - distillation_loss: 9.6809e-04
Epoch 7/60
3725/3725 [==============================] - 30s 8ms/step - categorical_accurac

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 0s 3ms/step - categorical_accuracy: 0.8290 - student_loss: 0.7798


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
print(classification_report(y_test_dt, preds_student_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.929329  0.952899  0.940966       276
           1   0.222222  0.052632  0.085106        38
           2   0.717703  0.937500  0.813008       160
           3   0.870370  0.770492  0.817391        61
           4   0.796610  0.594937  0.681159        79

    accuracy                       0.828990       614
   macro avg   0.707247  0.661692  0.667526       614
weighted avg   0.807486  0.828990  0.808949       614



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.95289855, 0.05263158, 0.9375    , 0.7704918 , 0.59493671])

In [ ]:
#60 epochs teacher

In [ ]:
teacher_model=model_b(verbose=VBS) #60 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 12s 8ms/step - loss: 0.9506 - accuracy: 0.8502


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.95289855, 0.10526316, 0.95      , 0.67213115, 0.78481013])

In [ ]:
print(classification_report(y_test_dt, preds_teacher_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.956364  0.952899  0.954628       276
           1   0.333333  0.105263  0.160000        38
           2   0.748768  0.950000  0.837466       160
           3   0.953488  0.672131  0.788462        61
           4   0.765432  0.784810  0.775000        79

    accuracy                       0.850163       614
   macro avg   0.751477  0.693021  0.703111       614
weighted avg   0.838857  0.850163  0.835298       614



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=70)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/70
3725/3725 [==============================] - 33s 8ms/step - categorical_accuracy: 0.7003 - student_loss: 0.7316 - distillation_loss: 0.0016
Epoch 2/70
3725/3725 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7681 - student_loss: 0.5597 - distillation_loss: 0.0012
Epoch 3/70
3725/3725 [==============================] - 28s 8ms/step - categorical_accuracy: 0.7840 - student_loss: 0.5226 - distillation_loss: 0.0012
Epoch 4/70
3725/3725 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7941 - student_loss: 0.5018 - distillation_loss: 0.0011
Epoch 5/70
3725/3725 [==============================] - 28s 8ms/step - categorical_accuracy: 0.7989 - student_loss: 0.4884 - distillation_loss: 0.0011
Epoch 6/70
3725/3725 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8046 - student_loss: 0.4744 - distillation_loss: 0.0011
Epoch 7/70
3725/3725 [==============================] - 28s 8ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 0s 3ms/step - categorical_accuracy: 0.8404 - student_loss: 0.7854


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
print(classification_report(y_test_dt, preds_student_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.939286  0.952899  0.946043       276
           1   0.250000  0.052632  0.086957        38
           2   0.728155  0.937500  0.819672       160
           3   0.881356  0.852459  0.866667        61
           4   0.803279  0.620253  0.700000        79

    accuracy                       0.840391       614
   macro avg   0.720415  0.683148  0.683868       614
weighted avg   0.818354  0.840391  0.820401       614



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.95289855, 0.05263158, 0.9375    , 0.85245902, 0.62025316])

In [ ]:
cm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

In [ ]:
cm.diagonal()

array([0.95289855, 0.05263158, 0.9375    , 0.85245902, 0.62025316])

In [ ]:
#50 epochs teacher model

#alpha=0.9, temp=4

#50 epochs: 82.41
#60 epochs: 84.53
#70 epochs: 84.04

#alpha=0.9, temp=5

#50 epochs: 81.43
#60 epochs: 83.88
#70 epochs: 

#alpha=0.9, temp=3

#50 epochs: 83.88
#60 epochs: 83.06


Trying out diff teachers and different students for perspective(the teacher who's epochs had been tuned)

In [ ]:
final_teacher_model=model_b(verbose=VBS) #60 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
final_teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
final_teacher_model.fit(x_dt_rshp, y_dt_, epochs=60)

Epoch 1/60
3725/3725 [==============================] - 45s 8ms/step - loss: 0.6851 - accuracy: 0.7238
Epoch 2/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.4053 - accuracy: 0.8412
Epoch 3/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.3292 - accuracy: 0.8688
Epoch 4/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2831 - accuracy: 0.8881
Epoch 5/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2532 - accuracy: 0.9002
Epoch 6/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2305 - accuracy: 0.9087
Epoch 7/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.2154 - accuracy: 0.9146
Epoch 8/60
3725/3725 [==============================] - 29s 8ms/step - loss: 0.1998 - accuracy: 0.9208
Epoch 9/60
3725/3725 [==============================] - 30s 8ms/step - loss: 0.1895 - accuracy: 0.9250
Epoch 10/60
3725/3725 [==============================] - 29s 8ms/step - l

In [ ]:
accuracy_final_teacher_model = final_teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 1s 5ms/step - loss: 0.9913 - accuracy: 0.8502


In [ ]:
preds_final_teacher_model = final_teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_final_teacher_model_fin=np.argmax(preds_final_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_final_teacher = confusion_matrix(y_test_dt, preds_final_teacher_model_fin)
matrix_final_teacher.diagonal()/matrix_final_teacher.sum(axis=1)

array([0.94202899, 0.18421053, 0.95      , 0.67213115, 0.78481013])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_final_teacher = f1_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
prec_final_teacher = precision_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
recall_final_teacher = recall_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_final_teacher)) 
    print("Recall: {}".format(recall_final_teacher))
    print("F1 score: {}".format(f1_final_teacher))

    report_final_teacher = classification_report(y_test_dt, preds_final_teacher_model_fin, digits=6)
    print(report_final_teacher)

Precision: 0.7645464077284243
Recall: 0.7066361571892597
F1 score: 0.7224304917801438
              precision    recall  f1-score   support

           0   0.962963  0.942029  0.952381       276
           1   0.350000  0.184211  0.241379        38
           2   0.748768  0.950000  0.837466       160
           3   0.976190  0.672131  0.796117        61
           4   0.784810  0.784810  0.784810        79

    accuracy                       0.850163       614
   macro avg   0.764546  0.706636  0.722430       614
weighted avg   0.847603  0.850163  0.841347       614



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
final_student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=final_student_model, teacher=final_teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=70)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/70
3725/3725 [==============================] - 32s 8ms/step - categorical_accuracy: 0.6921 - student_loss: 0.7538 - distillation_loss: 0.0047
Epoch 2/70
3725/3725 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7635 - student_loss: 0.5765 - distillation_loss: 0.0037
Epoch 3/70
3725/3725 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7826 - student_loss: 0.5312 - distillation_loss: 0.0034
Epoch 4/70
3725/3725 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7940 - student_loss: 0.5042 - distillation_loss: 0.0032
Epoch 5/70
3725/3725 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8009 - student_loss: 0.4887 - distillation_loss: 0.0031
Epoch 6/70
3725/3725 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8065 - student_loss: 0.4734 - distillation_loss: 0.0030
Epoch 7/70
3725/3725 [==============================] - 29s 8ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 0s 3ms/step - categorical_accuracy: 0.8469 - student_loss: 0.7093


In [ ]:
preds_final_student_model = final_student_model.predict(x_test_dt_rshp)

In [ ]:
preds_final_student_model_fin=np.argmax(preds_final_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_final_student = confusion_matrix(y_test_dt, preds_final_student_model_fin)
matrix_final_student.diagonal()/matrix_final_student.sum(axis=1)

array([0.96376812, 0.05263158, 0.9375    , 0.80327869, 0.67088608])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_final_student = f1_score(y_test_dt, preds_final_student_model_fin, average="macro")
prec_final_student = precision_score(y_test_dt, preds_final_student_model_fin, average="macro")
recall_final_student = recall_score(y_test_dt, preds_final_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_final_student)) 
    print("Recall: {}".format(recall_final_student))
    print("F1 score: {}".format(f1_final_student))

    report_final_student = classification_report(y_test_dt, preds_final_student_model_fin, digits=6)
    print(report_final_student)

Precision: 0.7408203991130821
Recall: 0.6856128918726709
F1 score: 0.6890084652490482
              precision    recall  f1-score   support

           0   0.926829  0.963768  0.944938       276
           1   0.333333  0.052632  0.090909        38
           2   0.750000  0.937500  0.833333       160
           3   0.890909  0.803279  0.844828        61
           4   0.803030  0.670886  0.731034        79

    accuracy                       0.846906       614
   macro avg   0.740820  0.685613  0.689008       614
weighted avg   0.824522  0.846906  0.825532       614

